In [2]:
import requests
import json
import time
import pandas as pd

In [ ]:
api_key = 'rjj-CYij9D5rBs0P7jLtSGO6u2hBp38A5jMZTGELqjvvojBqfepQac4jprE6LhD84IuHE9UuswLpOL46Ou8IA04Jd4Ije97e5rv5Q4Ifag8tQrMlJirC2-eTUH58ZXYx'
headers = {'Authorization': 'Bearer ' + api_key}

In [26]:
def get_all_business_ids(base_url):
    
    all_business_ids = []

    while True:
        response = requests.get(base_url, headers=headers)
        if response.status_code != 200:
            break  

        data = response.json()
        businesses = data.get('businesses', [])
        if not businesses:
            break  # Break the loop if no more businesses are returned

        for business in businesses:
            business_id = business.get('id')
            if business_id:
                all_business_ids.append(business_id)

        # Update the offset in the URL for the next request
        if 'offset=' in base_url:
            base_url = base_url.rsplit('offset=', 1)[0] + f'offset={len(all_business_ids)}'
        else:
            base_url += f'&offset={len(all_business_ids)}'

        time.sleep(1)  

    return all_business_ids


In [27]:
base_url = (
    'https://api.yelp.com/v3/businesses/search?'
    'location=New+York+City&term=restaurants&categories=french'
    '&price=3&price=4&sort_by=best_match&limit=50'
)
nyc_restaurant_ids = get_all_business_ids(base_url)
print(nyc_restaurant_ids)

['ETgJqJHV7BW6pIr9Ox74sA', 'DGhWO1sUWydVeR5j5ZZaMw', 'KgpOYAG-r_eDsQXFXt0nnQ', 'ZjU9qd-RR55cEEKufmdN5Q', 'FiRB9Ek3Xo9JHwXgBYITYg', 's3jou_L_LVYGkNHiuhjlew', '8XHbZIKDQbXhyKIGiukVeQ', 'npiIhpcWePbVT3JYEcy3zg', 'gtXP0CmOnBbgfnff7VXa7A', 'PflHKSowWSsR-a76Ne7nzg', 'EF4M55USa1ZNy1rP5Kuk2w', 'V3dHB6xAdd8_dH7t481PAw', 'TQzGf4k3HXyMbnNkYKKQSw', 'wDLHOgux_G8Y82Y6D9rSog', 'PYcL2z9gHj6-ZTlViT9Nqw', 'P2CTp4xyaNCe2AXSiLeOSQ', 'cHvd_fnNqbcR4EdyodzGzw', 'aTFkjo0scqmOgsFF5kuGXg', 'obFs12x7mgYPGxgalBd22w', 'nBkSqOhZMa1A7Yk3q8VPaA', 'CcxreLJbZsO6Z775-hMWpw', 'mxDfu4bHwSXoCr3FRpyMsw', 'C6f_MQaRwIbJ8diailVaCA', 'g5f6p9jfzB30Mhcu6d5OWA', 'GBJOu9Rk4JqqWjB8ddYudw', 'tmzWZtu9Nimn7Hfas9XmcA', 'HJg9TbpyWvb6-AFPdb7NCA', 'qLL_x4uzDHCgXd9lwaGxjw', 'OMv26_R8kaek1G7_QhBPkg', 'mjGdtrh0tX7I91GM9dlMNg', 'q-QEwZi32JCnjb7zbY-PQw', 'kMljfja7weqfl_lzspjzHA', '7N6Jle8ovxzAiiwwtE77Yw', 'NMVRHrkdp68twl4CWo1I5A', 'FON3TceJNsTC0I0mzBwOKg', 'DZaN3bNWfwcm__sNw5bvDg', '50fWXrNECQQYght4ZWeBXQ', 'sBL7J7s4Rnul40-MB0hXbA', 'UJe9_p6NEl

In [28]:
len(nyc_restaurant_ids)

97

In [29]:
def get_reviews(restaurant_ids,city):
    list_of_reviews = []
    for i in range(len(restaurant_ids)):
        url2 = "https://api.yelp.com/v3/businesses/" + restaurant_ids[i] + "/reviews?limit=50"
        #print(url2)
        response = requests.get(url2, headers=headers)
        reviews_data = response.json()
        
        try:
            for review in reviews_data['reviews']:
                review_dict = {'text': review['text'], 'rating': review['rating'],'location':city}
                list_of_reviews.append(review_dict)
        except:
            print("No reviews for this restaurant")
        
    return list_of_reviews


In [30]:
nyc_list_of_reviews = get_reviews(nyc_restaurant_ids,'New York City')

### San Francisco

In [31]:
sf_url = "https://api.yelp.com/v3/businesses/search?location=San+Francisco&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

san_francisco_restaurants = get_all_business_ids(sf_url)

sf_list_of_reviews = get_reviews(san_francisco_restaurants,'San Francisco')

No reviews for this restaurant


In [32]:
print(len(sf_list_of_reviews))

75


### Miami

In [33]:
miami_url = "https://api.yelp.com/v3/businesses/search?location=Miami&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

miami_restaurants = get_all_business_ids(miami_url)

miami_list_of_reviews = get_reviews(miami_restaurants,'Miami')

In [34]:
print(len(miami_list_of_reviews))

15


### Los Angeles

In [35]:
los_angeles_url = "https://api.yelp.com/v3/businesses/search?location=Los+Angeles&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

los_angeles_restaurants = get_all_business_ids(los_angeles_url)

los_angeles_list_of_reviews = get_reviews(los_angeles_restaurants,'Los Angeles')

In [36]:
print(len(los_angeles_list_of_reviews))

93


#### Merge

In [49]:
# Assuming the lists are named los_angeles_list_of_reviews, new_york_list_of_reviews, etc.
df1 = pd.DataFrame(los_angeles_list_of_reviews, columns=['text', 'rating', 'location'])
df2 = pd.DataFrame(nyc_list_of_reviews, columns=['text', 'rating', 'location'])
df3 = pd.DataFrame(sf_list_of_reviews, columns=['text', 'rating', 'location'])
df4 = pd.DataFrame(miami_list_of_reviews, columns=['text', 'rating', 'location'])
# Add more dataframes as needed

# Concatenate all the dataframes
df = pd.concat([df1, df2,df3,df4], ignore_index=True)

# Save the dataframe to a CSV file
df.to_csv('yelp_reviews.csv', index=False)

In [51]:
df['rating'].value_counts()

rating
5    267
4    104
3     59
2     25
1     15
Name: count, dtype: int64

In [52]:
df

,text,rating,location
0,Robyn gave amazing service! So attentive and f...,5,Los Angeles
1,Headed downtown on a Thursday evening for a Ki...,5,Los Angeles
2,"Been here a few times, in just recent weeks. T...",4,Los Angeles
3,Service is fast. Staff is friendly. The food i...,5,Los Angeles
4,Walked by and asked to see a menu. Very helpfu...,3,Los Angeles
...,...,...,...
465,Food amazing! Service and integrity 0. Dined ...,1,Miami
466,We were fortunate enough to be invited to the ...,5,Miami
467,"I've dined here twice, both times for a work r...",5,Miami
468,This is two reviews at once having been there ...,4,Miami


## Data Cleaning

In [46]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from textblob import Word
from collections import Counter
from nltk.stem import WordNetLemmatizer

In [47]:
df = pd.read_csv('yelp_reviews.csv')

In [48]:
df.drop_duplicates(inplace=True)
df.dropna(subset=['text', 'rating', 'location'], inplace=True)

Do we need to to the spelling correction before ot after the tokenization ?? <br>

In [49]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase
    lowercase_text = text.lower()

    # Remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)

    # Tokenization
    tokens = word_tokenize(lowercase_text)

    # Remove punctuation and stop words
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

df['text'] = df['text'].astype(str)  # Convert the column to string
df['text'] = df['text'].apply(lambda x: str(TextBlob(x)))  # Apply TextBlob to each element

df['tokens'] = df['text'].apply(preprocess_text)
#df['tokens'] = df['tokens'].apply(lambda x: [Word(word).spellcheck() for word in x])  


In [50]:
# Word Frequency Analysis
all_words = [word for tokens in df['tokens'] for word in tokens]
word_freq = Counter(all_words)

# N-gram Analysis
bigrams = ngrams(all_words, 2)
bigram_freq = Counter(bigrams)

# Example: Display most common words and bigrams
print(word_freq.most_common(10))
print(bigram_freq.most_common(10))

[('food', 139), ('service', 106), ('restaurant', 102), ('place', 79), ('great', 77), ('french', 71), ('came', 67), ('dinner', 62), ('good', 57), ('time', 51)]
[(('french', 'onion'), 18), (('onion', 'soup'), 18), (('new', 'york'), 14), (('food', 'service'), 13), (('service', 'great'), 12), (('first', 'time'), 10), (('amazing', 'service'), 9), (('dining', 'experience'), 9), (('service', 'excellent'), 9), (('french', 'restaurant'), 9)]
